# Stage 1: Preprocess VNG's data
In this stage, we will read raw data from a given dataset. The dataset consists of variable-resolution images, while our system requires a constant input dimensionality. Therefore, we need to down-sampled the images to a fixed resolution (270 x 270)

## Examples of processing
 + In the bellow code, we will crop the central region of raw image.

In [ ]:
%matplotlib inline
import glob
import os
import numpy as np
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt
import tensorflow as tf

raw_image = imread('model/datasets/nudity_dataset/3.jpg')

# Define a tensor placeholder to store an image
image = tf.placeholder("uint8", [None, None, 3])
image1 = tf.image.convert_image_dtype(image, dtype=tf.float32)
image2 = tf.image.central_crop(image1, central_fraction=0.875) # Crop the central region of raw image

model = tf.initialize_all_variables() # Quan trong

print raw_image.shape
with tf.Session() as session:
    session.run(model)
    result = session.run(image2, feed_dict={image: raw_image})
    print result.dtype
    print("The shape of result: ",result.shape)

print result.shape
## Draw image
fig = plt.figure()
a = fig.add_subplot(1,2,1)
plt.imshow(raw_image)
a = fig.add_subplot(1,2,2)
plt.imshow(result)
plt.show()


+ In the code bellow, resize image into the special resolution

In [ ]:
import numpy as np
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt
import tensorflow as tf

raw_image = imread('model/datasets/nudity_dataset/3.jpg')

image = tf.placeholder("uint8", [None, None, 3])
image1 = tf.image.convert_image_dtype(image, dtype = tf.float32)
image1_t = tf.expand_dims(image1, 0)
image2 = tf.image.resize_bilinear(image1_t, [270, 270], align_corners=False)
image2 = tf.squeeze(image2, [0])

image3 = tf.sub(image2, 0.5)
image3 = tf.mul(image2, 2.0)

model = tf.initialize_all_variables()

with tf.Session() as session:
    session.run(model)
    result = session.run(image3, feed_dict={image:raw_image})

## Draw image
fig = plt.figure()
a = fig.add_subplot(1,2,1)
plt.imshow(raw_image)
a = fig.add_subplot(1,2,2)
plt.imshow(result)
plt.show()


1.1 Create a standard training dataset

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import tensorflow  as tf
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
from model.datasets.data import generate_standard_dataset

# Load Normal and Nude images into the train dataset

image_normal_ls, file_name_normal = generate_standard_dataset('/home/cpu11757/workspace/Nudity_Detection/src/model/datasets/train/normal')

nudity_ls, file_name_nudity = generate_standard_dataset('/home/cpu11757/workspace/Nudity_Detection/src/model/datasets/train/nude')

init_op = tf.initialize_all_variables()

labels = np.zeros(3000, dtype = np.uint)


database = []
with tf.Session() as session:
    session.run(init_op)
    # Start populating the filename queue
    coord = tf.train.Coordinator()
    tf.train.start_queue_runners(coord=coord)    
    for i in range(3000):
        #print i
        if i % 2 == 0:
            image = image_normal_ls.eval()
        else:
            image = nudity_ls.eval()
            labels[i] = 1
        database.append(image)
    coord.request_stop()

database = np.array(database)


In [7]:
from Dataset.data import generate_standard_dataset
import numpy as np
import tensorflow as tf


img_nudity, _ = generate_standard_dataset('/media/taivu/Data/Project/Nudity_Detection/src/model/datasets/AdditionalDataset/vng/sex')

labels = np.ones(100, dtype = np.uint)

dataset = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    tf.train.start_queue_runners(coord=coord)
    
    for i in range(100):
        image = img_nudity.eval()
        dataset.append(image)
    coord.request_stop()
    
database = np.array(dataset)
    

In [ ]:
print file_name_normal[1123]

+ Generate tfrecords

In [8]:
import os
import tensorflow as tf

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def convert_to(data_dir, dataset, labels, name):
    """Converts a dataset to tfrecords."""
    images = dataset
    labels = labels
    num_examples = dataset.shape[0]
    
    rows, cols, depth = dataset[0].shape
    
    filename = os.path.join(data_dir, name + '.tfrecords')
    
    writer = tf.python_io.TFRecordWriter(filename)
    
    for idx in range(num_examples):
        image_raw = images[idx].tostring()
        example = tf.train.Example(features = tf.train.Features(feature={
                    'height': _int64_feature(rows),
                    'width': _int64_feature(cols),
                    'depth': _int64_feature(depth),
                    'label': _int64_feature(int(labels[idx])),
                    'image_raw': _bytes_feature(image_raw)
                }))
        writer.write(example.SerializeToString())
    writer.close()

convert_to('/home/taivu/workspace/NudityDetection/Dataset',
          database, labels, 'nudity_test_set')


+  Read a batch images

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
    serialized_example,
    features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
            'depth': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64)
        })
    
    image = tf.decode_raw(features['image_raw'], tf.float32)
    image = tf.reshape(image,[34,34,3])
    label = tf.cast(features['label'], tf.int32)
    height = tf.cast(features['height'], tf.int32)
    width = tf.cast(features['width'], tf.int32)
    depth = tf.cast(features['depth'], tf.int32)
    return image, label, height, width, depth

def data_input(data_dir, batch_size):
    filename_queue = tf.train.string_input_producer([data_dir], num_epochs = None)
    image, label, height, width, depth = read_and_decode(filename_queue)
    
    images_batch, labels_batch = tf.train.shuffle_batch(
        [image, label], 
        batch_size = batch_size,
        capacity = 2000,
        min_after_dequeue = 80
    )
    return images_batch, labels_batch


#filename_queue = tf.train.string_input_producer(['/home/cpu11757/workspace/Nudity_Detection/src/model/datasets/vng_dataset.tfrecords'], num_epochs = None)
#image, label, height,_,depth = read_and_decode(filename_queue)
img_batch, lb_batch = data_input('/home/cpu11757/workspace/Nudity_Detection/src/model/datasets/vng_dataset.tfrecords',500)

init_op = tf.initialize_all_variables()
fig = plt.figure()

with tf.Session() as session:
    session.run(init_op)
    coord = tf.train.Coordinator()
    tf.train.start_queue_runners(coord=coord)
    images, labels = session.run([img_batch, lb_batch])
    coord.request_stop()


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.imshow(images[1])
print labels[0]
plt.show()

+ Example shuffle dataset

In [ ]:
import tensorflow as tf

f = ["f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8"]
l = ["l1", "l2", "l3", "l4", "l5", "l6", "l7", "l8"]

fv = tf.constant(f)
lv = tf.constant(l)

rsq = tf.RandomShuffleQueue(10, 0, [tf.string, tf.string], shapes=[[],[]])
do_enqueues = rsq.enqueue_many([fv, lv])

gotf, gotl = rsq.dequeue()

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    coord = tf.train.Coordinator()
    tf.train.start_queue_runners(sess=sess,coord = coord)
    sess.run(do_enqueues)
    for i in xrange(2):
        one_f, one_l = sess.run([gotf, gotl])
        print "F: ", one_f, "L: ", one_l
    
    coord.request_stop()



+ Example cPickle

In [ ]:
import cPickle as pickle

dict1 = {'name':[],'id':[]}
dict2 = {'local':[], 'paza':[]}

#with open('test.p', 'wb') as fp:
#    pickle.dump(dict1,fp)
#    pickle.dump(dict2,fp)
    
with open('test.p', 'rb') as fp:
    d1 = pickle.load(fp)
    d2 = pickle.load(fp)

print len(d1)
print len(d2)
    

+ Example reshape

In [ ]:
import tensorflow as tf
import numpy as np

a = tf.constant(np.array([[.1]]))
init = tf.initialize_all_variables()
with tf.Session() as session:
    session.run(init)
    b = session.run(tf.nn.softmax(a))
    c = session.run(tf.nn.softmax_cross_entropy_with_logits([0.6, 0.4],[0,1]))
    #print b
    #print c

label = np.array([[0], [1], [1]])
idx = np.arange(3) * 2
print ('IDX')
print idx

labels_one_hot = np.zeros((3,2))
print ('labels_one_hot')
print labels_one_hot


labels_one_hot.flat[idx + label.ravel()] = 1
print ('IDX + label.ravel()')
print idx + label.ravel()


In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
from Dataset.data import preprocess_image
import numpy as np

filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once(
    '/home/taivu/workspace/NudityDetection/Dataset/train/normal/*.jpg'))

img_reader = tf.WholeFileReader()

_, img_file = img_reader.read(filename_queue)

image = tf.image.decode_jpeg(img_file, 3)

image = preprocess_image(image, 34, 34)

images = tf.train.batch([image],
                       batch_size = 10,
                       capacity = 50,
                       name = 'input')

coord = tf.train.Coordinator()


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(coord=coord)
    
    result_img = sess.run([images])
    
    result_img = np.array(result_img)
    
    coord.request_stop()
    coord.join(threads)

fig = plt.figure()
plt.imshow(result_img[0][1])
plt.show()
    
    

In [6]:
import tensorflow as tf
import numpy as np
from execute_model import evaluate
from Dataset.data import data_input
import matplotlib.pyplot as plt

dt, _ = data_input('/home/taivu/workspace/NudityDetection/Dataset/vng_dataset_validation.tfrecords', 10, False)
coord = tf.train.Coordinator()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(coord=coord)
    
    result_img = sess.run([dt])
    coord.request_stop()
    coord.join(threads)

#fig = plt.figure()
result_img = np.array(result_img)
print result_img.shape
print result_img.dtype
#plt.show()
    

(1, 10, 34, 34, 3)
float32


In [5]:
import tensorflow as tf
import numpy as np
from execute_model import evaluate
from Dataset.data import data_input
import matplotlib.pyplot as plt

dt = data_input('/home/taivu/workspace/NudityDetection/Dataset/vng_dataset_validation.tfrecords', 10, False, False)
coord = tf.train.Coordinator()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(coord=coord)
    
    result_img = sess.run([dt])
    coord.request_stop()
    coord.join(threads)

#fig = plt.figure()
result_img = np.array(result_img)
print result_img.shape
print result_img.dtype
#plt.show()
    

(1, 10, 34, 34, 3)
float32


In [7]:
import tensorflow as tf
import os
import glob
from Dataset.data import preprocess_image
import matplotlib.pyplot as plt

data_dir = '/home/taivu/workspace/AddPic'

filenames = []
for pathAndFilename in glob.iglob(os.path.join(data_dir, '*.jpg')):
    filenames.append(pathAndFilename)
    
filename_queue = tf.train.string_input_producer(filenames, shuffle = None)

filename = filename_queue.dequeue()

# img_reader = tf.WholeFileReader()

img_file = tf.read_file(filename)

#_, img_file = img_reader.read(filename)

img = tf.image.decode_jpeg(img_file, 3)

img = preprocess_image(img, 34, 34)

filename_batch, img_batch = tf.train.batch([filename, img], batch_size = 3, capacity=200, name = 'input')

init = tf.global_variables_initializer()

coord =tf.train.Coordinator()

with tf.Session() as sess:
    sess.run(init)
    tf.train.start_queue_runners(sess, coord)    
    ls_img, ls_nf = sess.run([img_batch, filename_batch])
    
#fig = plt.figure()

print ls_nf
for i in range(3):
    a = fig.add_subplot(1,3, i)
    a.set_title('%d'%i)
    plt.imshow(ls_img[i])
plt.show()


    
coord.request_stop()

['/home/taivu/workspace/AddPic/b20.jpg'
 '/home/taivu/workspace/AddPic/d8.jpg' '/home/taivu/workspace/AddPic/8.jpg']INFO:tensorflow:Error reported to Coordinator: <type 'exceptions.RuntimeError'>, Attempted to use a closed Session.



In [8]:
print ls_nf[0]

/home/taivu/workspace/AddPic/b20.jpg


In [6]:
import tensorflow as tf
import numpy as np

a = [[1,2,3]]
b = [[4,5,6]]
np.column_stack((a,b))

array([[1, 2, 3, 4, 5, 6]])

In [9]:
import math

print int(math.ceil(float(5)/3))

2
